In [46]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

### Load data

In [18]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test = tf.keras.utils.normalize(x_test, axis=1)

### Define model architecture

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

### Train the model

In [31]:
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)
model.fit(x_train, y_train, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0310 - accuracy: 0.9896
Epoch 2/3
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0232 - accuracy: 0.9927
Epoch 3/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0197 - accuracy: 0.9933


### Evaluate the model

In [30]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Loss: {test_loss} \tAcc: {test_acc * 100:.2f}%')

313/313 [==============================] - 0s 863us/step - loss: 0.4456 - accuracy: 0.9009
Loss: 0.4455881714820862 	Acc: 90.09%


### Save model

In [33]:
tf.saved_model.save(model, "../saved_models/")

INFO:tensorflow:Assets written to: ../saved_models/assets


INFO:tensorflow:Assets written to: ../saved_models/assets


### Convert to Tflite

In [48]:
converter = tf.lite.TFLiteConverter.from_saved_model("../saved_models/MNIST")
tflite_model = converter.convert()

2022-12-21 08:31:34.163499: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-21 08:31:34.163526: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-21 08:31:34.163660: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ../saved_models/MNIST
2022-12-21 08:31:34.164897: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-21 08:31:34.164915: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ../saved_models/MNIST
2022-12-21 08:31:34.168340: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-12-21 08:31:34.204289: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ../saved_models/MNIST
2022-12-21 08:31:34.212596: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: O

### Save converted model

In [36]:
with open('../saved_models/MNIST_Lite/model.tflite', 'wb') as f:
  f.write(tflite_model)

### Testing model on example input

In [47]:
pred = model.predict(x_test[0].reshape(-1, 28*28))

print(np.argmax(pred))

1/1 [==============================] - 0s 17ms/step
7


In [65]:
tf.keras.utils.save_img('../saved_models/MNIST_Lite/test_input.jpg', x_test[0].reshape((28, 28, 1)))

In [75]:
im = plt.imread('../saved_models/MNIST_Lite/test_input.jpg')
im = tf.keras.utils.normalize(im.reshape((28, 28)))

In [78]:
np.argmax(model.predict(im.reshape(-1, 28*28)))

1/1 [==============================] - 0s 22ms/step


7